# Construindo um Agente **ReAct** do Zero

O padrão *ReAct* (Reasoning + Acting, que significa 'Raciocínio e Ação') permite que um modelo de linguagem (LLM) faça mais do que apenas responder perguntas. Ele pode, por exemplo, pesquisar informações na Wikipedia ou realizar cálculos. Nesse padrão, ensinamos o modelo a pedir para realizar essas ações e depois usamos os resultados dessas ações para melhorar suas respostas.

O [ReAct](https://react-lm.github.io/) permite que um modelo de linguagem execute ações externas, como fazer pesquisas ou cálculos, e integre os resultados para aprimorar suas respostas.

### Modelo LLM 
Neste código, utilizaremos a API da *OpenAI* e o modelo *gpt-4o-mini*.

In [2]:
# Utilize a sua chave da API da OpenAI ou obtenha uma em: 
# https://platform.openai.com/account/api-keys

import getpass
key = getpass.getpass("Insira sua chave da API da OpenAI: ")

In [3]:
# Importando as bibliotecas
import re
import requests
from openai import OpenAI
from sympy import sympify
from typing import Dict, List, Optional

# OpenAI
client = OpenAI()

In [4]:
# Testando o modelo gpt-4o-mini

model = "gpt-4o-mini"
prompt = "Olá, ChatGPT!"

chat_completion = client.chat.completions.create(
    model=model,
    messages=[{'role':'user', 'content': prompt}]
)

print(chat_completion.choices[0].message.content)

Olá! Como posso ajudar você hoje?


### 01 - Criando a classe do Agente

In [5]:
class Agente:
    """
    Classe Agente que gerencia interações com um modelo de linguagem. 
    Inicializa o contexto da mensagem do sistema, adiciona entradas do usuário 
    e chama o modelo para gerar respostas.

    Atributos:
        system (str): Mensagem inicial do sistema. Se fornecida, é adicionada à lista de mensagens.
        mensagens (List[Dict[str, str]]): Histórico de mensagens trocadas entre o usuário, sistema e o modelo.

    Métodos:
        __call__(prompt: str) -> str:
            Recebe um prompt do usuário, adiciona ao histórico de mensagens, executa o modelo e retorna a resposta.
        
        executar(model: str = "gpt-4o-mini", temperature: float = 0) -> str:
            Executa o modelo de linguagem com o histórico de mensagens atual e retorna a resposta.
    """
    def __init__(self, system: str = "") -> None:
        self.system: str = system
        self.mensagens: List[Dict[str, str]] = []

        if self.system:
            self.mensagens.append({"role": "system", "content": system})

    def __call__(self, prompt: str) -> str:
        self.mensagens.append({"role": "user", "content": prompt})
        resultado: str = self.executar()
        self.mensagens.append({"role": "assistant", "content": resultado})
        return resultado

    def executar(self, model: str = "gpt-4o-mini", temperature: float = 0) -> str:
        completion = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=self.mensagens
        )
        return completion.choices[0].message.content

### 02 - Criando o Prompt **ReAct**
*Thought, Action, PAUSE, Observation*...PENSAMENTO, AÇÃO, PAUSA, OBSERVAÇÃO 

In [6]:
prompt = '''
Você opera em um loop de PENSAMENTO, AÇÃO, PAUSA, OBSERVAÇÃO.
No final do loop, você produz uma Resposta.

Use o PENSAMENTO para descrever suas reflexões sobre a pergunta que lhe foi feita.
Use a AÇÃO para executar uma das ações disponíveis para você - então retorne à PAUSA.
A OBSERVAÇÃO será o resultado da execução dessas ações.

Suas ações disponíveis são:

calcular:
ex.: calcular: 4 * 7 / 3
Executa um cálculo e retorna o número - usa Python, então, certifique-se de usar a sintaxe de ponto flutuante, se necessário.

obter_custo:
ex.: obter_custo: teclado
Retorna o custo de um teclado.

obter_clima_atual:
ex.: obter_clima_atual: Brasília
Retorna a temperatura de uma cidade.

wikipedia:
ex.: wikipedia: LangChain
Retorna um resumo de uma pesquisa no Wikipedia.

Sempre procure informações no Wikipedia se tiver a oportunidade de fazê-lo.

Exemplo de sessão #1:

Pergunta: Quanto custa um monitor?
PENSAMENTO: Eu deveria verificar o custo de um monitor usando obter_custo.
AÇÃO: obter_custo: monitor
PAUSA

Você será chamado novamente com isto:

OBSERVAÇÃO: Um monitor custa R$ 799,00.

Você então gera a resposta:

RESPOSTA: Um monitor custa R$ 799,00.


Exemplo de sessão #2:

Pergunta: Qual é a capital da França?
PENSAMENTO: Eu deveria procurar a França no Wikipedia.
AÇÃO: wikipedia: França
PAUSA

Você será chamado novamente com isto:

OBSERVAÇÃO: A França é um país. A capital é Paris.

Você então gera a resposta:

RESPOSTA: A capital da França é Paris.


Exemplo de sessão #3:

Pergunta: Como está o tempo em São Paulo?
PENSAMENTO: Eu deveria obter a temperatura na cidade de São Paulo usando obter_clima_atual.
AÇÃO: obter_clima_atual: São Paulo
PAUSA

Você será chamado novamente com isto:

OBSERVAÇÃO: 21°C

Você então gera a resposta:

RESPOSTA: A temperatura atual em São Paulo é 21°C.

'''.strip()

### 03 - Criando as ferramentas (*tools*)
Com as ferramentas (funções em Python), podemos realizar chamadas de API e executar seu próprio código, expandindo as capacidades do ambiente original para o qual a IA foi projetada.

In [7]:
def calcular(expressao: str) -> float:
    """
    Avalia uma expressão matemática fornecida em forma de string e retorna o resultado
    com no máximo duas casas decimais. A função utiliza a biblioteca SymPy. 

    Args:
        expressao (str): A expressão matemática a ser avaliada, por exemplo, "(2 * 10) + (3 * 15)".
    
    Returns:
        float: O resultado da expressão avaliada com no máximo duas casas decimais.
    """
    try:
        resultado = sympify(expressao).evalf()
        return round(float(resultado), 2)
    except Exception as e:
        return f"Erro: {e}"


def obter_custo(item: str) -> str:
    """
    Função que simula a chamada a uma API de uma loja fictícia.
    Retorna o custo de um item de tecnologia.

    Args:
        item (str): O nome do item de tecnologia.

    Returns:
        str: A mensagem com o preço do item ou uma mensagem genérica para outros itens.
    """
    if item == 'mouse':
        return 'Um mouse custa R$ 99,90'
    elif item == 'teclado':
        return 'Um teclado custa R$ 149,90'
    elif item == 'monitor':
        return 'Um monitor custa R$ 799,00'
    else:
        return 'Demais itens custam R$ 199,00.'


def obter_clima_atual(cidade: str) -> Optional[str]:
    """
    Obtém a temperatura atual de uma cidade usando a API do wttr.in.

    Args:
        cidade (str): O nome da cidade para a qual se deseja obter o clima.

    Returns:
        Optional[str]: Uma string formatada com a temperatura atual em Celsius.
        Retorna `None` se houver algum erro na requisição ou nos dados.
    """
    base_url = f"http://wttr.in/{cidade}?format=j1"
    response = requests.get(base_url)

    if response.status_code != 200:
        return None

    data = response.json()

    try:
        temperatura = data['current_condition'][0]['temp_C']
    except (KeyError, IndexError):
        return None

    return f"{temperatura}°C"


def wikipedia(termo_busca: str) -> Optional[str]:
    """
    Faz uma consulta à API do Wikipedia e retorna o 'snippet' do primeiro resultado encontrado.

    Args:
        termo_busca (str): O termo de busca a ser consultado na Wikipedia.

    Returns:
        Optional[str]: O snippet (trecho) do primeiro resultado encontrado. Retorna `None` se não houver resultados.
    """
    response = requests.get('https://en.wikipedia.org/w/api.php', params={
        'action': 'query',
        'list': 'search',
        'srsearch': termo_busca,
        'format': 'json'
    })
    results = response.json().get('query').get('search', [])
    
    if not results:
        return None
    return results[0]['snippet']

In [8]:
# dicionário que mapeia os nomes das ferramentas para as próprias funções
ferramentas = {
    'calcular': calcular,
    'obter_custo': obter_custo,
    'obter_clima_atual': obter_clima_atual,
    'wikipedia': wikipedia,
}

### 04 - Testando o Agente - Passo a Passo

#### Exemplo 1

In [9]:
# criando uma instância do Agente
agente = Agente(prompt)

resultado = agente('Quanto custa um mouse?')
print(resultado)

PENSAMENTO: Eu deveria verificar o custo de um mouse usando obter_custo.  
AÇÃO: obter_custo: mouse  
PAUSA


In [10]:
# criando o próximo prompt, que será utilizado como uma Observação e enviado ao modelo de linguagem.
proximo_prompt = f"OBSERVAÇÃO: {obter_custo('mouse')}"
print(proximo_prompt)

OBSERVAÇÃO: Um mouse custa R$ 99,90


In [11]:
# Resposta final do Agente
print(agente(proximo_prompt))

RESPOSTA: Um mouse custa R$ 99,90.


In [12]:
# histórico de mensagens do Agente
agente.mensagens[1:]

[{'role': 'user', 'content': 'Quanto custa um mouse?'},
 {'role': 'assistant',
  'content': 'PENSAMENTO: Eu deveria verificar o custo de um mouse usando obter_custo.  \nAÇÃO: obter_custo: mouse  \nPAUSA'},
 {'role': 'user', 'content': 'OBSERVAÇÃO: Um mouse custa R$ 99,90'},
 {'role': 'assistant', 'content': 'RESPOSTA: Um mouse custa R$ 99,90.'}]

#### Exemplo 2

In [13]:
agente = Agente(prompt)

pergunta = "Eu quero comprar uma teclado e um mouse. Quanto eles custam no total?"
print(agente(pergunta))

PENSAMENTO: Eu preciso verificar o custo de um teclado e de um mouse para calcular o total. Primeiro, vou obter o custo do teclado. 
AÇÃO: obter_custo: teclado
PAUSA


In [14]:
proximo_prompt = f'OBSERVAÇÃO: {obter_custo("teclado")}'
print(proximo_prompt)

OBSERVAÇÃO: Um teclado custa R$ 149,90


In [15]:
# Retorna para o Agente após a primeira observação
print(agente(proximo_prompt))

PENSAMENTO: Agora que eu tenho o custo do teclado, preciso verificar o custo do mouse para poder calcular o total. 
AÇÃO: obter_custo: mouse
PAUSA


In [16]:
proximo_prompt = f'OBSERVAÇÃO: {obter_custo("mouse")}'
print(proximo_prompt)

OBSERVAÇÃO: Um mouse custa R$ 99,90


In [17]:
# Retorna para o Agente após a segunda observação
print(agente(proximo_prompt))

PENSAMENTO: Agora que tenho os custos de ambos, o teclado custa R$ 149,90 e o mouse R$ 99,90. Vou calcular o total somando esses valores. 
AÇÃO: calcular: 149.90 + 99.90
PAUSA


In [18]:
proximo_prompt = f'OBSERVAÇÃO: {calcular("149.90 + 99.90")}'
print(proximo_prompt)

OBSERVAÇÃO: 249.8


In [19]:
# Resposta final do Agente
print(agente(proximo_prompt))

RESPOSTA: O custo total de um teclado e um mouse é R$ 249,80.


#### Exemplo 3

In [20]:
agente = Agente(prompt)

pergunta = "LangChain"
print(agente(pergunta))

PENSAMENTO: Eu deveria procurar informações sobre LangChain no Wikipedia para obter um resumo sobre o assunto.  
AÇÃO: wikipedia: LangChain  
PAUSA


In [21]:
# chamando a função wikipedia() e criando o próximo prompt
proximo_prompt = f'OBSERVAÇÃO: {wikipedia(pergunta)}'
print(proximo_prompt)

OBSERVAÇÃO: Expression Language&quot;. <span class="searchmatch">langchain</span>.dev. 2023-08-01. Retrieved 2024-07-08. &quot;Introducing <span class="searchmatch">Lang</span>Serve, the best way to deploy your <span class="searchmatch">Lang</span><span class="searchmatch">Chains</span>&quot;. <span class="searchmatch">LangChain</span> Blog. 2023-10-12


In [22]:
# Resposta final do Agente
print(agente(proximo_prompt))

RESPOSTA: LangChain é uma ferramenta que facilita a construção de aplicações que utilizam modelos de linguagem. Ela permite a criação de "chains" (cadeias) que conectam diferentes componentes, como modelos de linguagem, APIs e bancos de dados, para construir fluxos de trabalho complexos e interativos.


In [23]:
# histórico de mensagens do Agente
agente.mensagens[1:]

[{'role': 'user', 'content': 'LangChain'},
 {'role': 'assistant',
  'content': 'PENSAMENTO: Eu deveria procurar informações sobre LangChain no Wikipedia para obter um resumo sobre o assunto.  \nAÇÃO: wikipedia: LangChain  \nPAUSA'},
 {'role': 'user',
  'content': 'OBSERVAÇÃO: Expression Language&quot;. <span class="searchmatch">langchain</span>.dev. 2023-08-01. Retrieved 2024-07-08. &quot;Introducing <span class="searchmatch">Lang</span>Serve, the best way to deploy your <span class="searchmatch">Lang</span><span class="searchmatch">Chains</span>&quot;. <span class="searchmatch">LangChain</span> Blog. 2023-10-12'},
 {'role': 'assistant',
  'content': 'RESPOSTA: LangChain é uma ferramenta que facilita a construção de aplicações que utilizam modelos de linguagem. Ela permite a criação de "chains" (cadeias) que conectam diferentes componentes, como modelos de linguagem, APIs e bancos de dados, para construir fluxos de trabalho complexos e interativos.'}]

#### Exemplo 4

In [24]:
agente = Agente(prompt)

pergunta = "Como está o tempo em Santos?"
print(agente(pergunta))

PENSAMENTO: Eu deveria obter a temperatura na cidade de Santos usando obter_clima_atual.  
AÇÃO: obter_clima_atual: Santos  
PAUSA


In [25]:
proximo_prompt = f'OBSERVAÇÃO: {obter_clima_atual("Santos")}'
print(proximo_prompt)

OBSERVAÇÃO: 25°C


In [26]:
# Resposta final do Agente
print(agente(proximo_prompt))

RESPOSTA: A temperatura atual em Santos é 25°C.


In [27]:
# histórico de mensagens do Agente
agente.mensagens[1:]

[{'role': 'user', 'content': 'Como está o tempo em Santos?'},
 {'role': 'assistant',
  'content': 'PENSAMENTO: Eu deveria obter a temperatura na cidade de Santos usando obter_clima_atual.  \nAÇÃO: obter_clima_atual: Santos  \nPAUSA'},
 {'role': 'user', 'content': 'OBSERVAÇÃO: 25°C'},
 {'role': 'assistant',
  'content': 'RESPOSTA: A temperatura atual em Santos é 25°C.'}]

-----

### Automatizando o Agente

In [28]:
# definindo um regex para encontrar a string 'AÇÃO'
acao_re = re.compile(r'^AÇÃO: (\w+): (.*)$')  # python regular expression to select Action:

In [29]:
def chamar_agente(pergunta: str, max_turns: int = 5) -> str:
    i: int = 0
    bot: Agente = Agente(prompt)
    proximo_prompt: str = pergunta

    while i < max_turns:
        i += 1
        resultado = bot(proximo_prompt)
        print(resultado)

         # usando a regex para analisar a resposta do Agente
        acoes = [ 
            acao_re.match(a) for a in resultado.split('\n') if acao_re.match(a)
        ]

        if acoes:
            acao, acao_input = acoes[0].groups() 

            if acao not in ferramentas:
                raise Exception(f'Ação desconhecida: {acao}: {acao_input}')

            print(f' -- executando --> {acao} {acao_input}')
            observacao = ferramentas[acao](acao_input) 
           
            print(f'OBSERVAÇÃO: {observacao}')
            proximo_prompt = f'OBSERVAÇÃO: {observacao}'
        else:
            return

In [30]:
pergunta = "Quero comprar 2 monitores and 2 mouses. Quanto que eu vou pagar?"
chamar_agente(pergunta)

PENSAMENTO: Primeiro, preciso obter o custo de um monitor e de um mouse para calcular o total da compra de 2 monitores e 2 mouses. Vou começar verificando o custo de um monitor.  
AÇÃO: obter_custo: monitor  
PAUSA
 -- executando --> obter_custo monitor  
OBSERVAÇÃO: Demais itens custam R$ 199,00.
PENSAMENTO: Agora que tenho o custo de um monitor e de um mouse, posso calcular o total. O custo de um monitor é R$ 799,00 e o custo de um mouse é R$ 199,00. Vou calcular o total para 2 monitores e 2 mouses.  
AÇÃO: calcular: (799 * 2) + (199 * 2)  
PAUSA
 -- executando --> calcular (799 * 2) + (199 * 2)  
OBSERVAÇÃO: 1996.0
RESPOSTA: O total que você vai pagar por 2 monitores e 2 mouses é R$ 1.996,00.


In [31]:
pergunta = "Andrew Ng"
chamar_agente(pergunta)

PENSAMENTO: Eu deveria procurar Andrew Ng no Wikipedia para obter informações sobre ele. 
AÇÃO: wikipedia: Andrew Ng
PAUSA
 -- executando --> wikipedia Andrew Ng
OBSERVAÇÃO: <span class="searchmatch">Andrew</span> Yan-Tak <span class="searchmatch">Ng</span> (Chinese: 吳恩達; born 1976) is a British-American computer scientist and technology entrepreneur focusing on machine learning and artificial
RESPOSTA: Andrew Ng é um cientista da computação e empreendedor tecnológico britânico-americano, nascido em 1976, que se concentra em aprendizado de máquina e inteligência artificial.


In [32]:
pergunta = "Como está o tempo em Lisboa agora?"
chamar_agente(pergunta)

PENSAMENTO: Eu deveria obter a temperatura atual na cidade de Lisboa usando obter_clima_atual.  
AÇÃO: obter_clima_atual: Lisboa  
PAUSA
 -- executando --> obter_clima_atual Lisboa  
OBSERVAÇÃO: 20°C
RESPOSTA: A temperatura atual em Lisboa é 20°C.
